<a href="https://colab.research.google.com/github/waveology/aire/blob/main/4_caracterizaci%C3%B3n_estadistica_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caracterización estadística de los datos

* Vamos a explorar algunas de las posibilidades de análisis de datos de calidad del aire que ofrece Python en ámbito de la estadística. 
* Usaremos como hasta ahora datos meteorológicos de AEMET y registros de contaminación de la Comunidad de Madrid.

###1. Preparación de datos y código
---

* Para trabajar más cómodamente vamos a descargar todos los ficheros del repositorio GitHub
* De esa manera dispondremos de los datos localmente para nuestro análisis.

In [1]:
# Directorio de trabajo local en la nube de Google Colab 
# ------------------------------------------------------------
%cd /content

# Si existe una copia previa del repositorio, la borramos:
# ----------------------------------------------------------------------------
!  rm -rf aire

# Creamos una copia del repositorio SOLO si no existe previamente
# ----------------------------------------------------------------------------
! [ ! -d aire ] && git clone https://github.com/waveology/aire.git

# Entramos en el repositorio que acabamos de copiar
# --------------------------------------------------
%cd aire

/content
Cloning into 'aire'...
remote: Enumerating objects: 291, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 291 (delta 118), reused 131 (delta 82), pack-reused 82
Receiving objects: 100% (291/291), 19.67 MiB | 17.71 MiB/s, done.
Resolving deltas: 100% (156/156), done.
/content/aire


* Importamos las extensiones que vamos a necesitar. 

* Para simplificar la tarea hemos empaquetado las funciones de lectura de datos en un fichero independiente llamado "lectura_de_datos.py"

In [3]:
import lectura_de_datos                                   # lee ficheros de datos meteorológicos y de contaminación de Madrid
import matplotlib.pyplot as plt                           # dibujo de gráficos
from matplotlib.dates import MonthLocator, DateFormatter  # formato de fechas 
from scipy import stats                                   # cálculo estadístico
import numpy as np                                        # matrices

###2. Inventario de datos
---

* Por conveniencia, hemos introducido en "lectura_de_datos.py" el inventario de estaciones de medida de la contaminación.
* Listamos los campos: **código | municipio | nombre**

In [5]:
lectura_de_datos.estalist_com

{28005002: (5, 'ALCALÁ DE HENARES'),
 28006004: (6, 'ALCOBENDAS'),
 28007004: (7, 'ALCORCÓN'),
 28009001: (9, 'ALGETE'),
 28013002: (13, 'ARANJUEZ'),
 28014002: (14, 'ARGANDA DEL REY'),
 28016001: (16, 'EL ATAZAR'),
 28045002: (45, 'COLMENAR VIEJO'),
 28047002: (47, 'COLLADO VILLALB A'),
 28049003: (49, 'COSLADA'),
 28058004: (58, 'FUENLABRADA'),
 28065014: (65, 'GETAFE'),
 28067001: (67, 'GUADALIX DE LA SIERRA'),
 28074007: (74, 'LEGANÉS'),
 28080003: (80, 'MAJADAHONDA'),
 28092005: (92, 'MÓSTOLES'),
 28102001: (102, 'ORUSCO DE TAJUÑA'),
 28120001: (120, 'PUERTO DE COTOS'),
 28123002: (123, 'RIVAS-VACIAMADRID'),
 28133002: (133, 'SAN MARTÍN DE VALDEIGLESIAS'),
 28148004: (148, 'TORREJÓN DE ARDOZ'),
 28161001: (161, 'VALDEMORO'),
 28171001: (171, 'VILLA DEL PRADO'),
 28180001: (180, 'VILLAREJO DE SALVANÉS')}

* Listamos los contaminantes registrados.
* **Código  |  Magnitud  | Unidades**


In [6]:
lectura_de_datos.maglist_com

{1: ('Dióxido de azufre', 'μg/m³'),
 6: ('Monóxido de carbono', 'mg/m³'),
 7: ('Monóxido de nitrógeno', 'μg/m³'),
 8: ('Dióxido de nitrógeno', 'μg/m³'),
 9: ('Partículas en suspensión < PM2.5', 'μg/m³'),
 10: ('Partículas en suspensión < PM10', 'μg/m³'),
 12: ('Óxidos de nitrógeno', 'μg/m³'),
 14: ('Ozono', 'μg/m³'),
 20: ('Tolueno', 'μg/m³'),
 22: ('Black Carbon', 'μg/m³'),
 30: ('Benceno', 'μg/m³'),
 42: ('Hidrocarburos totales', 'mg/m³'),
 44: ('Hidrocarburos no metánicos', 'mg/m³'),
 431: ('MetaParaXileno', 'μg/m³')}

* Hacemos lo mismo con los datos meteorológicos
* Las estaciones (en este caso son las mismas)
* **código | municipio | nombre**


In [7]:
lectura_de_datos.estalist_meteo

{28005002: (5, 'ALCALÁ DE HENARES'),
 28006004: (6, 'ALCOBENDAS'),
 28007004: (7, 'ALCORCÓN'),
 28009001: (9, 'ALGETE'),
 28013002: (13, 'ARANJUEZ'),
 28014002: (14, 'ARGANDA DEL REY'),
 28016001: (16, 'EL ATAZAR'),
 28045002: (45, 'COLMENAR VIEJO'),
 28047002: (47, 'COLLADO VILLALB A'),
 28049003: (49, 'COSLADA'),
 28058004: (58, 'FUENLABRADA'),
 28065014: (65, 'GETAFE'),
 28067001: (67, 'GUADALIX DE LA SIERRA'),
 28074007: (74, 'LEGANÉS'),
 28080003: (80, 'MAJADAHONDA'),
 28092005: (92, 'MÓSTOLES'),
 28102001: (102, 'ORUSCO DE TAJUÑA'),
 28120001: (120, 'PUERTO DE COTOS'),
 28123002: (123, 'RIVAS-VACIAMADRID'),
 28133002: (133, 'SAN MARTÍN DE VALDEIGLESIAS'),
 28148004: (148, 'TORREJÓN DE ARDOZ'),
 28161001: (161, 'VALDEMORO'),
 28171001: (171, 'VILLA DEL PRADO'),
 28180001: (180, 'VILLAREJO DE SALVANÉS')}

* Y por último, la magnitudes meteorológicas
* **Código | Magnitud | Unidades**

In [8]:
lectura_de_datos.maglist_meteo

{81: ('Velocidad del viento', 'm/s'),
 82: ('Dirección del viento', 'º'),
 83: ('Temperatura', 'ºC'),
 86: ('Humedad relativa', '%'),
 87: ('Presión atmosférica', 'hPa'),
 88: ('Radiación solar', 'W/m2'),
 89: ('Precipitación', 'mm')}

###3. Carga de datos
---

* En el siguiente ejemplo, vamos a cargar datos de concentración de NO2 en la ciudad de Móstoles a lo largo del año 2019: 
* Esta versión de la funcion lectura_de_datos devuelve:
  * el dataframe con los datos
  * el nombre de la estación
  * la magnitud medida
  * sus unidades
* Eso facilita la automatización de los gráficos.



In [ ]:
anio = 2022
df, magnitud, unidades,estacion = lectura_de_datos.contaminacion(
                                      'datos/contaminacion/%s.csv' % anio,
                                      codigo_magnitud = 8,         # Dióxido de azufre
                                      codigo_estacion = 28092005    # Móstoles  
                                      ) 

#df, magnitud, unidades,estacion = lectura_de_datos.meteo(
#                                    'datos/meteo/%s.csv' % anio,
#                                     codigo_magnitud = 83,         # Temperatura
#                                     codigo_estacion = 28067001    # Guadalix de la Sierra  
#                                     ) 

print(df)

###4. Representación gráfica
---

Representamos los resultados en un gráfico simple X-Y

In [ ]:
ax = df.plot(marker='o',                                      # Símbolo
        ms=1,                                                 # Tamaño del símbolo
        lw=0,                                                 # Grosor de líneas de conexión
        color='black',                                        # Color
        grid=True,                                            # Rejilla
        figsize=(10,8),                                       # Tamaño del gráfico
        legend=False,                                         # Leyenda
        title='%s  -  %s     %s %s' % (estacion,anio,magnitud,unidades),  # Título
        #xlim=('2022-03-10','2022-04-01'),
        #ylim=(0,700)
        )

# Control de las etiquetas de tiempo
# ------------------------------------
#ax.xaxis.set_major_locator(MonthLocator())                             # Poner tics mayores al inicio de cada mes
#ax.xaxis.set_minor_locator(MonthLocator(bymonthday=15))                # poner tics menores el día 15 de cada mes
#ax.xaxis.set_major_formatter(DateFormatter("%d %B %y"))                # formato de la fecha para los tics mayores
#ax.xaxis.set_minor_formatter(DateFormatter('%d'))                      # formato de la fecha para los tics menores
#ax.tick_params(axis="x", labelrotation= 45)                            # inclinación de las etiquetas

###5. El histograma
---

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))
df.hist(  
         bins     =  range(0,175,5),
         stacked  =  False,
         density  =  False,    # ¿Número de casos o frecuencia estadística?
         log      =  False,    # Logarítimico
         rwidth   =  0.9,
         ax       =  ax
)
ax.set_title('Histograma    -    %s    -    %s    -     %s (%s)' % (anio,estacion,magnitud,unidades),size=14)
#ax.set_ylim(0,20)
ax.set_ylabel('Número de registros',size=15)
#ax.set_ylabel('Frecuencia estadística',size=15)
ax.set_xlabel(unidades,size=15)

###6. El valor medio
---

El valor medio de la serie temporal puede calcularse fácilmente con la función mean()

In [ ]:
media = df.mean()[0] 

Representamos graficamente:

In [ ]:
def dibuja(df, estadisticos) :
  
  fig, ax = plt.subplots(2,1,figsize=(12,7))

  min = df.min()
  max = df.max()

  # Serie temporal
  # --------------
  df.plot(grid=True,             ax = ax[0], color='#444444')
  txt = '%s    -    %s    -     %s (%s)     [min=%.1f,  max=%.1f]'
  ax[0].set_title(txt % (anio,estacion,magnitud,unidades,min,max),size=14)
  ax[0].set_ylabel(unidades,size=12)


  # Formato de fechas
  # -----------------
  ax[0].xaxis.set_major_locator(MonthLocator())                             # Poner tics mayores al inicio de cada mes
  ax[0].xaxis.set_minor_locator(MonthLocator(bymonthday=15))                # poner tics menores el día 15 de cada mes
  ax[0].xaxis.set_major_formatter(DateFormatter("%d %B %y"))                # formato de la fecha para los tics mayores
  ax[0].xaxis.set_minor_formatter(DateFormatter(''))                      # formato de la fecha para los tics menores
  ax[0].tick_params(axis="x", labelrotation= 45)                            # inclinación de las etiquetas

  # Histograma
  # -----------
  df.hist(bins = range(0,175,5), ax = ax[1], color='#444444')
  ax[1].set_title(None)
  ax[1].set_xlabel(unidades,size=12)

  # Añadimos líneas de valores estadísticos
  # -----------------------------------------
  color = ('b','r','g','c','orange','m','y','k')

  n = 0
  for nombre,valor in estadisticos.items() :
     ax[0].axhline(y=valor, ls='--', lw=3, label='%s=%.1f' % (nombre,valor), color=color[n])
     ax[1].axvline(x=valor, ls='--', lw=3,label='%s=%.1f' % (nombre,valor), color=color[n])
     n += 1


  # Activamos la leyenda
  # --------------------
  ax[0].legend()

  # Ajusta visualmente los gráficos
  # --------------------------------
  plt.tight_layout()

  return


In [ ]:
esta = {'media': media}
dibuja(df,esta)

###7. **La dispersión**
---



In [ ]:
varianza          = df.var()[0]
desviacion_tipica = df.std()[0]

esta['s1'] = media - desviacion_tipica
esta['s2'] = media + desviacion_tipica
dibuja(df,esta)

del esta['s1']
del esta['s2']

###8. **La mediana**
---

In [ ]:
mediana = df.median()[0] 

esta['mediana'] = mediana
dibuja(df,esta)

###9. **Percentiles**
---

In [ ]:
nivel = (1, 5, 25, 75, 95, 99)
for x in nivel :
   label = 'P%d' % x
   esta[label] = np.percentile(df,x)
dibuja(df,esta)

###10. **Valores atípicos (outliers)**
---

Un método popular para determinar qué puntos son atípicos y eliminarlos de la muestra se basa en el ***rango intercuartílico*** (***RIQ***).

El ***RIQ***  es la diferencia entre los percentiles 75 y 25.

Si un registro se encuentra a una distancia de P25 o P75 que supera 1.5 veces el riq, se considera un **valor atípico leve**.

Si un registro se a una distancia de P25 o P75 que supera 3 veces el riq, se considera un **valor atípico extremo**.

In [ ]:
P25, P75 = np.percentile(df, (25 ,75))
RIQ = P75 - P25

print('P25=%.1f  P75=%.1f   RIQ=%.1f' % (P25, P75, RIQ))

In [ ]:
# Valores atípicos leves
# ----------------------
L1 = P25 - 1.5 * RIQ
L2 = P75 + 1.5 * RIQ
print('Los valores inferiones a %.1f son ATÍPICOS LEVES' % L1)
print('Los valores superiores a %.1f son ATÍPICOS LEVES' % L2)

In [ ]:
# Valores atípicos extremos
# -------------------------
E1 = P25 - 3 * RIQ
E2 = P75 + 3 * RIQ
print('Los valores inferiones a %.1f son ATÍPICOS LEVES' % L1)
print('Los valores superiores a %.1f son ATÍPICOS LEVES' % L2)

In [ ]:
esta = {'L1':L1,'L2':L2,'E1':E1,'E2':E2}
dibuja(df,esta)

Veamos qué ocurre si eliminamos los valores atípicos extremos:

In [ ]:
df1 = df[ (df.valor >= E1) & (df.valor <= E2) ]
n = len(df) - len(df1)
print('Se eliminaron %d valores atípicos extremos de una muestra de %d datos (%.2f%%)' % (n,len(df),100*n/len(df)))

In [ ]:
esta = {'media': df1.mean()[0],'mediana': df1.median()[0]}
dibuja(df1,esta)

esta = {'media': df.mean()[0],'mediana': df.median()[0]}
dibuja(df,esta)

Si hubiéramos eliminado todos los valores atípicos:

In [ ]:
df1 = df[ (df.valor >= L1) & (df.valor <= L2) ]
n = len(df) - len(df1)
print('Se eliminaron %d valores atípicos leves de una muestra de %d datos (%.2f%%)' % (n,len(df),100*n/len(df)))

In [ ]:
esta = {'media': df1.mean()[0],'mediana': df1.median()[0]}
dibuja(df1,esta)